In [ ]:
%matplotlib widget

In [ ]:
import gymnasium as gym
import argparse
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import torch

torch.set_num_threads(4)

from hw4_agent import PolicyNetwork
from hw4_utils import preprocess_observation

args = argparse.Namespace(
    env="ALE/MsPacman-v5", mode="train", total_frames=1000000, batch_size=8, 
    unroll_length=80, hidden_dim=256, discounting=0.99, learning_rate=1e-4, 
    grad_norm_clipping=10.0, save_path=None, load_path=None, min_to_save=5, 
    eval_every=10, render=True)

env = gym.make(args.env)
naction = env.action_space.n
args.start_nlives = env.ale.lives()
model = PolicyNetwork(naction, args)
# load model parameters here...
#checkpoint = torch.load(load_path)
# etc

fig = plt.figure()
obs = env.reset(seed=590060)[0]
im = plt.imshow(obs, interpolation='None', animated=True)
observation = preprocess_observation( # 1 x 1 x ic x iH x iW
    obs).unsqueeze(0).unsqueeze(0)
prev_state = None

# save observations and play them back
model.eval()
val_imgs = []
nsteps = 200
for i in range(nsteps):
    action, prev_state = model.get_action(observation, prev_state)
    env_output = env.step(action)
    img = env_output[0]
    val_imgs.append(img)
    observation = preprocess_observation(
            env_output[0]).unsqueeze(0).unsqueeze(0)

def show_frame(thing):
    im.set_data(thing)
    return im,

anim = FuncAnimation(fig, show_frame, interval=50, frames=val_imgs, blit=True, repeat=False)